<a href="https://colab.research.google.com/github/abelowska/mlNeuro/blob/main/2025/MLN_signal_processing_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Signal processing in MNE

[`MNE`](https://mne.tools/stable/index.html) is an open-source Python package for exploring, visualizing, and analyzing human neurophysiological data: MEG, EEG, sEEG, ECoG, NIRS, and more.

The easiest way is to install MNE via Anaconda, `pip`, or `conda` (see [installation instructions](https://mne.tools/stable/install/manual_install.html)).

In [ ]:
!pip install mne
!pip install mne-bids
!pip install openneuro-py

Imports

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import os
import os.path as op

from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids
import os
import mne
import openneuro
from scipy import signal

## Download dataset

In [ ]:
# Dataset and subject information
dataset = "ds003775"
subject = "sub-001"

# Define the download path
bids_root = os.path.join(os.path.dirname(mne.datasets.sample.data_path()), dataset)

# Ensure the directory exists
os.makedirs(bids_root, exist_ok=True)

# Download the dataset (only the specified subject)
openneuro.download(dataset=dataset, target_dir=bids_root, include=subject)

## Load EEG data

In [ ]:
# BIDS parameters for loading EEG data
datatype = "eeg"
subject = "001"         # BIDS-formatted subject ID (without "sub-" prefix)
session = "t1"          # Session name (e.g., "ses-t1")
task = "resteyesc"      # Task name extracted from the file name
suffix = "eeg"          # Data type suffix (e.g., "eeg")

# Create a BIDSPath object
bids_path = BIDSPath(root=bids_root, datatype=datatype,
                     subject=subject, session=session,
                     task=task, suffix=suffix)

**MNE**-Python data structures are based around the FIF file format from Neuromag, but there are reader functions for a wide [variety of other data formats](https://mne.tools/stable/overview/implementation.html#data-formats). In our dataset data is stored in [The European Data Format (EDF) format](https://en.wikipedia.org/wiki/European_Data_Format). Data is loaded into so-called [`Raw`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw) object.


In [ ]:
# Load the raw EEG data with preloading enabled
raw = read_raw_bids(bids_path=bids_path, verbose=True,
                         extra_params={'preload': True})

# set the montage (localization of channels)
raw = raw.set_montage('biosemi64')

## Display basic information about the loaded `Raw` data

You can get a glimpse of the basic details of a Raw object by printing it; even more is available by printing its `info` attribute (a dictionary-like object that is preserved across Raw, Epochs, and Evoked objects). The `info` data structure keeps track of channel locations, applied filters, projectors, etc. Notice especially the chs entry, showing that MNE-Python detects different sensor types and handles each appropriately. See The Info data structure for more on the [`Info`](https://mne.tools/stable/generated/mne.Info.html) class.

In [ ]:
raw.info

In [ ]:
print(raw.info)

Let's see our EEG data. Basic MNE classes ([`Raw`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw), [`Epochs`](https://mne.tools/stable/generated/mne.Epochs.html), [`Evoked`](https://mne.tools/stable/generated/mne.Evoked.html)) have special method for plotting. Just call method `plot()` on the `Raw` object to see the data.

In [ ]:
fig = raw.plot()

In [ ]:
print(raw.info)

## Working with data

1. `Raw` store data as ndarray of `(n_channels, n_timepoints)` shape

In [ ]:
raw_data = raw.get_data()
print(raw_data)
print(f'\nRaw data shape: {raw_data.shape}')

### Exercise 1
You know the data shape and the sampling frequency. Calculate the length of the recorded signal in seconds.

### Exercise 2

Power Spectral Density (PSD) describes how the power (or variance) of a signal is distributed across different frequencies. It provides insight into the strength of the signal’s components at various frequencies, making it a useful tool for analyzing signals like EEG, audio, and other time-series data.

**Below there is a plot of PSD for our resting-state sample. Can you read from it the frequency of the power line?**

In [ ]:
fig = raw.compute_psd(fmin=0, fmax=100).plot()

**Can you tell from the PSD plot which brain frequencies are most pronounced in our resting-state sample?**

## Signal processing - filters

We have discussed filters and their main classes. Below are several examples of filter implementations. Some of them are integrated into `MNE`, while others are implemented using external libraries such as `scipy` or `numpy`.

### 1. Static filters

In [ ]:
# get data
raw_signal = raw.get_data()

# square data
squared_raw_signal = np.square(raw_signal)

# create new Raw with the squared data
squared_raw = mne.io.RawArray(squared_raw_signal, raw.info)

# plot squared Raw
fig = squared_raw.plot(scalings=dict(eeg=20e-7))

You can do this in many ways:

In [ ]:
# apply squaring function to Raw
squared_raw = raw.copy().apply_function(lambda x: np.square(x))

# plot squared Raw
fig = squared_raw.plot(scalings=dict(eeg=20e-7))

#### Exerice 3

Calculate the variance of the signal.

In [ ]:
# your code here

### 2. Spatial filters

Re-referencing is commonely used in signal pre-processing. In fact, re-referencing is an example of spatial filter: new signal is a linear combination of the signal from the rest of electrodes.

Imagine that our aim is to re-reference the signal to an average of the signal from all electrodes. It can be written as follow:

$Y(n) = X(n) - \frac{1}{m}\sum_{i=0}^{i=m}x_i(n)$

where $m$ is the number of channels.

#### Exercise 4

Try to implement re-referencing to the average on your own. Use `get_data()`, then calculate the average of all channels per timepint using `np.mean()` with correct `axis` parameter. Then you can iterate over channels and create new channels with re-referenced signal.

In [ ]:
raw_signal = raw.get_data()

# your code here

Re-referencing is also implemented in MNE:

In [ ]:
raw_rereferenced = raw.copy().set_eeg_reference(ref_channels='average')

# compare original and re-referenced signal
fig = raw.plot()
fig = raw_rereferenced.plot()

You can also choose the channel for re-referencing:

In [ ]:
raw_rereferenced = raw.copy().set_eeg_reference(ref_channels=['Fp1'])

# compare original and re-referenced signal
fig = raw.plot()
fig = raw_rereferenced.plot()

### Temporal filters

One example of a temporal filter is the moving average. The equation for the simple moving average is as follows:

$T := y_i(n) = \frac{1}{m} \sum_{k=0}^{m-1} x_i (n-k)$

#### (Exercise 5)
Try implementing the moving average on your own.

In [ ]:
# your code here

## Spectral filters

Spectral filters are special cases of temporal filters. A commonly used filter in pre-processing is the band-pass filter, which limits the frequency range of the signal. Filters are implemented in `MNE` and can be easily used. Below, you can find the `MNE` code for a 1-30 Hz band-pass filter.

In [ ]:
l_freq = 1
h_freq = 30

filtered_raw = raw.copy().filter(
      picks=['eeg', 'eog'],
      l_freq=l_freq,
      h_freq=h_freq,
      method='iir',
      iir_params=None
      )

# plot and compare Raws
fig = raw.plot()
fig = filtered_raw.plot()

#### Exercise 6

Look into the `MNE` documentation and apply a Notch filter to remove the power line noise. Apply the notch filter to `filtered_raw` to have both the band-pass and notch filters applied.

In [ ]:
# your code here

## (Exercise 7: Signal processing final pipeline from lecture)

Try to implement the filter pipeline from the lecture that returns the amplitude of the alpha osscilations for each timepoint. The pipeline was as follows:

`band-pass (8-13 Hz)` -> `square` -> `moving average` -> `square root`

In [ ]:
# your code here

## Prediction function framework

Filters can be integrated into the prediction function framework, where part of the signal transformation is handled by the predictive function, and the other part is done using the filtering approach.

Usually, spectral filtering is performed with filters, while spatial filters are part of the prediction function, which is applied to the final signal.

#### Exercise 8
Below you have almost implemented the combined *filters + prediction function* solution for our problem. Try to finish it.

In [ ]:
def predict_func(X):
    """
    This function calculates the square root of the variance of the input signal X.

    Parameters:
    X : numpy.ndarray
        The input signal, which can be a 1D or 2D array. In the case of 2D, it represents multiple channels.

    Returns:
    numpy.ndarray
        The square root of the variance for each channel (or the entire signal if it's 1D).
    """
    # Calculate the variance of the signal
    variance = np.var(X, axis=-1)  # Variance along the last axis

    # Return the square root of the variance
    return np.sqrt(variance)

In [ ]:
# 1. Filter data with the band pass
l_freq = 8
h_freq = 13
filtered_raw = raw.copy().filter(
      picks=['eeg', 'eog'],
      l_freq=l_freq,
      h_freq=h_freq,
      method='iir',
      iir_params=None
      )

# 2. For each timepoint i get k=10 elements chunks of data (i-k) and run the prediction function
k=10
data = filtered_raw.get_data(picks='Fz') # only one channel
n = data.shape[-1] # number of samples (n_timepoints)

y_predicted = []

# your code here: iterate over samples, get chunks, and supply then to predict_func()